A Jupyter Notebook script for scraping data from the NCBI website
    
Need to Install

<ol>
  <li>pip install  BeautifulSoup</li>
  <li>pip install  pandas</li>
  <li>pip install  tqdm</li>
</ol>

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

tqdm.pandas()
def find_submited_by(acc_arg):
    Answer = dict()
    ACC  = str(acc_arg)
    URL = f'https://www.ncbi.nlm.nih.gov/sra/?term={ACC}'
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    div_results = soup.find_all('div',class_='sra-full-data')
    for num, i in enumerate(div_results):
        string=i.getText()
        string = string.replace(u'\xa0', u' ')
        if "Submitted by:" in string:
            Answer['ACC'] = ACC
            Answer['Submitted_by'] = string.split(':')[1]
            return string.split(':')[1]
def find_ID(acc_arg):
    Answer = dict()
    ACC  = str(acc_arg)
    URL = f'https://www.ncbi.nlm.nih.gov/sra/?term={ACC}'
    page = requests.get(URL)    
    soup1 = BeautifulSoup(page.content, 'html.parser') 
    div_results2 = soup1.find_all('div',class_='resc')
    for num, i in enumerate(div_results2):
        string=i.getText()
        string = string.replace(u'\xa0', u' ')
        if "ID:" in string:
            Answer['ACC'] = ACC
            Answer['ID'] = string.split(':')[1]
            return string.split(':')[1]
def find_submission_biosample_field(ID_arg):
    Answer = dict()
    ID     = str(ID_arg)
    URL   = f'https://www.ncbi.nlm.nih.gov/biosample?LinkName=sra_biosample&from_uid={ID}'
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')  
    div_results = soup.find_all('div',class_='docsum')
    for num, i in enumerate(div_results):
        string=i.getText()
        string = string.replace(u'\xa0', u'')
        if "Submission" in string:
            return string.split('Submission')[1].split(';')[0]

In [ ]:
df = pd.read_csv('./Data/sample_data.csv')
df['Submitted_by'] = df['Acc'].progress_apply(find_submited_by)
df['ID'] = df['Acc'].progress_apply(find_ID)
df['Biosample_Submission'] = df['ID'].progress_apply(find_submission_biosample_field)

In [ ]:
df = df[['Acc', 'ID', 'Center Names', 'Submitted_by', 'Biosample_Submission']]
df.head()

In [ ]:
df.to_csv('./Data_Result/sample_data_Scrape_Result.csv', encoding='utf-8')
df.to_csv('./Data/sample_data_Scrape_Result.csv', encoding='utf-8')